In [13]:
import numpy as np
import pandas as pd
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/coronavirusdataset/TimeAge.csv
/kaggle/input/coronavirusdataset/Case.csv
/kaggle/input/coronavirusdataset/Policy.csv
/kaggle/input/coronavirusdataset/Region.csv
/kaggle/input/coronavirusdataset/PatientInfo.csv
/kaggle/input/coronavirusdataset/SearchTrend.csv
/kaggle/input/coronavirusdataset/Weather.csv
/kaggle/input/coronavirusdataset/TimeProvince.csv
/kaggle/input/coronavirusdataset/TimeGender.csv
/kaggle/input/coronavirusdataset/SeoulFloating.csv
/kaggle/input/coronavirusdataset/Time.csv


In [14]:
patient = pd.read_csv('/kaggle/input/coronavirusdataset/PatientInfo.csv') #kaggle의 데이터 중 PatientInfo를 patient라는 명령어로 지정
region = pd.read_csv('/kaggle/input/coronavirusdataset/Region.csv') #kaggle의 데이터 중 Region을 Region라는 명령어로 지정

In [15]:
patient.head()

,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,male,50s,Korea,Seoul,Gangseo-gu,overseas inflow,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,male,30s,Korea,Seoul,Jungnang-gu,overseas inflow,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,male,50s,Korea,Seoul,Jongno-gu,contact with patient,2002000001,17,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,male,20s,Korea,Seoul,Mapo-gu,overseas inflow,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,female,20s,Korea,Seoul,Seongbuk-gu,contact with patient,1000000002,2,NaN,2020-01-31,2020-02-24,NaN,released


In [16]:
region.head()

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,Seoul,Gangdong-gu,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,Seoul,Gangbuk-gu,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,Seoul,Gangseo-gu,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080


In [17]:
regional_patient = pd.merge(patient[['patient_id','confirmed_date','sex','age','province','city']],region[['province','city','latitude','longitude']], how = 'left', on = ['province','city']) #Patient 데이터 내의 정보를 각 항목별로 분배
display(regional_patient.head())
display(regional_patient.isnull().sum() / regional_patient.shape[0])

,patient_id,confirmed_date,sex,age,province,city,latitude,longitude
0,1000000001,2020-01-23,male,50s,Seoul,Gangseo-gu,37.551166,126.849506
1,1000000002,2020-01-30,male,30s,Seoul,Jungnang-gu,37.606832,127.092656
2,1000000003,2020-01-30,male,50s,Seoul,Jongno-gu,37.572999,126.979189
3,1000000004,2020-01-30,male,20s,Seoul,Mapo-gu,37.566283,126.901644
4,1000000005,2020-01-31,female,20s,Seoul,Seongbuk-gu,37.589562,127.016700


patient_id        0.000000
confirmed_date    0.000581
sex               0.217231
age               0.267183
province          0.000000
city              0.018199
latitude          0.042401
longitude         0.042401
dtype: float64

In [18]:
import folium
from folium import plugins

import json

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

In [19]:
regional_count = regional_patient[['latitude','longitude']].dropna()

m = folium.Map([36, 128], zoom_start=8) # folium map 플러그인을 이용해 위도, 경도, 크기에 맞춰 지도를 불러옴

plugins.MousePosition().add_to(m) # 현재 마우스 포인터 위치를 읽어옴

plugins.MarkerCluster(regional_count).add_to(m) # folium map에 저장된 지역 경계를 읽어와 현재 마우스 포인터가 위치한 지역을 굵은 선으로 표기함

m

In [20]:
male_patient = regional_patient\
.query('sex in ("male")')[['latitude','longitude']].dropna()

female_patient = regional_patient\
.query('sex in ("female")')[['latitude','longitude']].dropna() # patient 데이터 내의 성별을 불러와 성별에 따른 위치를 지도에 표기함


m = folium.Map([36, 128], zoom_start=8) 

initial = folium.plugins.MarkerCluster(control=False)

m.add_child(initial)

male_g = plugins.FeatureGroupSubGroup(initial, '남성확진자') 
m.add_child(male_g)

female_g = plugins.FeatureGroupSubGroup(initial, '여성확진자')
m.add_child(female_g)


plugins.MarkerCluster(female_patient).add_to(female_g)

plugins.MarkerCluster(male_patient).add_to(male_g)



folium.LayerControl(collapsed=False).add_to(m)

m

In [21]:
heat_data = regional_patient\ # 기존의 환자의 숫자로 표기되던 것을 히트맵으로 표기함
.groupby(['latitude','longitude'])['patient_id'].count().reset_index()\
.values\
.tolist()

m = folium.Map([36, 128], zoom_start=8)#, tiles = 'stamentoner')

folium.plugins.HeatMap(heat_data).add_to(m)

m

In [22]:
regional_patient_bytime = regional_patient\
.groupby(['confirmed_date','latitude','longitude'])['patient_id'].count().reset_index()\
.sort_values(by = 'confirmed_date')

date_rng = pd.date_range(regional_patient_bytime['confirmed_date'].min(), regional_patient_bytime['confirmed_date'].max())
date_rng = [str(i)[:10] for i in date_rng]

all_date_region = pd.DataFrame({'confirmed_date' : np.repeat(date_rng,region[['latitude','longitude']].drop_duplicates().shape[0]),
              'latitude' : np.tile(list(region['latitude'].drop_duplicates()), len(date_rng)),
              'longitude' : np.tile(list(region['longitude'].drop_duplicates()), len(date_rng))})

all_date_region = pd.merge(all_date_region, regional_patient_bytime, on = ['confirmed_date','latitude','longitude'], how = 'left').fillna(0)
all_date_region['cumsum'] = all_date_region.groupby(['latitude','longitude'])['patient_id'].cumsum()

all_date_region = all_date_region[all_date_region['cumsum'] != 0]

time_index, data = [], []

for date in all_date_region['confirmed_date'].unique(): # 환자의 확진 일자별로 데이터를 나열함
    time_index += [date[6:].replace('-','/')]
    temp_list = all_date_region.query('confirmed_date == @date')[['latitude','longitude','cumsum']].values.tolist()
    data += [temp_list]
    del temp_list

In [23]:
data[:5], time_index[:5]

([[[37.551165999999995, 126.849506, 1.0]],
  [[37.551165999999995, 126.849506, 1.0]],
  [[37.551165999999995, 126.849506, 1.0]],
  [[37.551165999999995, 126.849506, 1.0], [37.658363, 126.831961, 1.0]],
  [[37.551165999999995, 126.849506, 1.0],
   [37.658363, 126.831961, 1.0],
   [36.992293, 127.11270900000001, 1.0]]],
 ['1/23', '1/24', '1/25', '1/26', '1/27'])

In [24]:
m = folium.Map([36, 128], zoom_start=8)#, tiles = 'Mapbox Bright')

hm = folium.plugins.HeatMapWithTime(data, index=time_index, auto_play=False, min_opacity=0.3, radius = 25, ) # 히트맵을 시간별로 재생할 수 있도록 설정

hm.add_to(m)

for bottom, click in zip([60,90,120,150,180],['Loop','Forward','Play','Reverse','Backward']): # 지도 하단에 데이터 재생 표를 생성
    
    
    name = '''
            <div style="position: fixed; bottom: '''+str(bottom)+'''px; left: 50px; width: 100px; height: 29px; 
                        background-color: white; border:1px solid grey; z-index:9999; font-size:11px;text-align:center;"
                        >&nbsp; <br><b>Click : '''+ click +'''</b></div>'''  

    m.get_root().html.add_child(folium.Element(name))

m